## Week 3 Final Exercise: Synthetic Dataset Generator using Open-Source Models
Generate structured Q&A / instruction datasets on any topic using open-source LLMs via OpenRouter, with an interactive Gradio UI.

In [ ]:
%pip install -q gradio openai python-dotenv pandas

In [1]:
import os, json
import pandas as pd
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
OPENROUTER_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

MODELS = [
    "meta-llama/llama-3.1-8b-instruct",
    "mistralai/mistral-7b-instruct",
    "google/gemma-2-9b-it:free",
    "qwen/qwen-2.5-7b-instruct",
]

DATASET_TYPES = ["Q&A", "Instruction-Response", "Classification"]

SYSTEM_PROMPT = """You are a synthetic dataset generator. Output ONLY a valid JSON array.
Each element must be an object with keys: \"input\" and \"output\".
No markdown, no explanation, no extra text — just the raw JSON array."""

In [3]:
def _build_prompt(topic, dataset_type, n):
    templates = {
        "Q&A": f"Generate {n} question-answer pairs about '{topic}'. Each object: {{\"input\": \"<question>\", \"output\": \"<answer>\"}}.",
        "Instruction-Response": f"Generate {n} instruction-response pairs for the domain '{topic}'. Each object: {{\"input\": \"<instruction>\", \"output\": \"<response>\"}}.",
        "Classification": f"Generate {n} text classification examples about '{topic}'. Each object: {{\"input\": \"<text>\", \"output\": \"<label>\"}}.",
    }
    return templates[dataset_type]

In [4]:
def generate_dataset(topic, dataset_type, n, model):
    client = OpenAI(base_url=OPENROUTER_URL, api_key=OPENROUTER_API_KEY)
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": _build_prompt(topic, dataset_type, n)},
        ],
        temperature=0.8,
    )
    raw = response.choices[0].message.content.strip()
    # strip markdown code fences if model wraps output
    if raw.startswith("```"):
        raw = raw.split("```")[1]
        if raw.startswith("json"):
            raw = raw[4:]
    return json.loads(raw)

In [5]:
def _to_dataframe(records):
    return pd.DataFrame(records)

def _save(records, fmt):
    df = _to_dataframe(records)
    path = f"synthetic_dataset.{fmt}"
    if fmt == "json":
        df.to_json(path, orient="records", indent=2)
    else:
        df.to_csv(path, index=False)
    return path

In [6]:
def run(topic, dataset_type, n, model, export_fmt):
    try:
        records = generate_dataset(topic, dataset_type, int(n), model)
        df = _to_dataframe(records)
        path = _save(records, export_fmt)
        return df, path, f"Generated {len(records)} records successfully."
    except Exception as e:
        return pd.DataFrame(), None, f"Error: {e}"

In [7]:
with gr.Blocks(title="Synthetic Dataset Generator", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Synthetic Dataset Generator\nPowered by open-source LLMs via OpenRouter")

    with gr.Row():
        with gr.Column(scale=1):
            topic     = gr.Textbox(label="Topic", placeholder="e.g. Python decorators, Climate change, Customer support")
            dtype     = gr.Dropdown(choices=DATASET_TYPES, value="Q&A", label="Dataset Type")
            n_records = gr.Slider(minimum=5, maximum=50, value=10, step=5, label="Number of Records")
            model     = gr.Dropdown(choices=MODELS, value=MODELS[0], label="Model")
            fmt       = gr.Radio(choices=["json", "csv"], value="json", label="Export Format")
            btn       = gr.Button("Generate", variant="primary")

        with gr.Column(scale=2):
            status  = gr.Textbox(label="Status", interactive=False)
            preview = gr.Dataframe(label="Preview")
            dl_file = gr.File(label="Download")

    btn.click(fn=run, inputs=[topic, dtype, n_records, model, fmt], outputs=[preview, dl_file, status])

demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
